# Vanilla RNN from Scratch for Vehicle Sensor Data

This notebook implements a vanilla RNN from scratch using only NumPy to process vehicle sensor data.


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
# Load vehicle sensor dataset
# Update the path to your dataset file
dataset_path = 'path/to/your/dataset.csv'  # ⬅️ CHANGE THIS to your actual file path

# Load the dataset (supports CSV, Excel, etc.)
if dataset_path.endswith('.csv'):
    df = pd.read_csv(dataset_path)
elif dataset_path.endswith('.xlsx') or dataset_path.endswith('.xls'):
    df = pd.read_excel(dataset_path)
else:
    # Try CSV first, if it fails try other formats
    df = pd.read_csv(dataset_path)

# Display basic information about the dataset
print(f"Dataset loaded successfully!")
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()


In [ ]:
# Explore the dataset structure
print("Dataset Info:")
print(df.info())

print("\n" + "="*50)
print("Basic Statistics:")
print(df.describe())

print("\n" + "="*50)
print("Check for missing values:")
print(df.isnull().sum())

print("\n" + "="*50)
print("Data types:")
print(df.dtypes)
